# Portfolio Optimization
The portfolio optimization module is a collection of functions that can be used to optimize a portfolio of assets. The module contains functions to calculate the efficient frontier, the optimal portfolio, the return of the optimal portfolio and more.

Prototyping and experimenting with this module should be easy and fun. The documentation is therefore written in the form of a tutorial with examples.

### Modules
The portfolio optimization module consists of the following modules:
- **data_processing**: This module contains functions to process data. For example, it contains functions to calculate the expected returns and the covariance matrix of a portfolio.
- **data_collection**: This module contains functions to collect data. For example, it contains functions to collect price data from CoinMetrics.
- **optimization**: This module contains functions to optimize a portfolio. For example, it contains functions to calculate the efficient frontier and the optimal portfolio.
- **portfolio**: This module contains objects to represent a portfolio. This is mainly used to store and rebalance a portfolio.
- **backtesting**: This module contains functions to backtest a portfolio.

In [1]:
# Importing data processing modules, such as expected returns.
from portfolio_optimization.data_processing import *
# Importing data collection modules, such as getting assets.
from portfolio_optimization.data_collection import *
# Importing optimization modules, such as the optimizer. This is the main module for generating portfolios.
from portfolio_optimization.optimization import *
# Importing portfolio modules, such as the portfolio class. This is the main module for using optimizers
from portfolio_optimization.portfolio import *
# Importing backtesting modules, such as the backtester. This is the main module for backtesting portfolios.
from portfolio_optimization.backtesting import *


## Note: The following imports are for the documentation only
from tokens.get_assets import *
import numpy as np
import pandas as pd
from datetime import timedelta
from dateutil.relativedelta import relativedelta

## Data Collection
The first step is to collect data. The data collection module contains functions to collect data from CoinMetrics. The data is fetched as a git repository and stored in a local directory. The data is then read of the CSV files in this directory (in our case the `data` directory). The raw data is converted to a Pandas DataFrame returned by `get_historical_prices_for_assets`. Since the data is containing a lot of information, we will only use the `ReferenceRate` and `CapMrktEstUSD` columns. The `ReferenceRate` column contains the price of the asset and the `CapMrktEstUSD` column contains the market cap of the asset. The market cap is used to calculate the weights of the assets in the black litterman portfolio.

In [2]:
# Choose the asset class and the assets
asset_list = get_tickers()
asset_class = "high_risk_tickers"

_df = get_historical_prices_for_assets(
    asset_list[asset_class],
    time_range=timedelta(days=365 * 1 + 120),  # 1 years
    interested_columns=["ReferenceRate", "CapMrktEstUSD"],
)

High risk tickers: 12
Medium risk tickers: 60
Low risk tickers: 8


/Users/arguiot/Developer/Pyratz/portfolio/portfolio_optimization/data_collection/get_crypto_price_range.py:41: DtypeWarning: Columns (146) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


## Data Processing
The data processing module is an internal module that is not going to be used by the frontend (Manager), here this is just some quick data cleaning for the notebook.

In [3]:
# Filter out all columns containing `_` in their name
df = _df.loc[:, ~_df.columns.str.contains("_")]

# Get all the market caps
mcaps = _df.loc[:, _df.columns.str.contains("CapMrktEstUSD")]
mcaps.columns = mcaps.columns.str.replace("_CapMrktEstUSD", "")
mcaps.replace(np.nan, 0, inplace=True)

start_date_portfolio = df.index[0] + relativedelta(days=120)

/var/folders/d2/1qh2h9ys19z3375prb_0cb8w0000gn/T/ipykernel_23799/3818043710.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mcaps.replace(np.nan, 0, inplace=True)


## Delegates & Data pipelines
`Portfolio` and `Optimization` modules were designed with pipeline architecture in mind. That means that some steps in each process can be delegated to external logic.

### Optimization
Delegating for an `GeneralOptimization` class is easy. You just need to extend `GeneralOptimizationDelegate`:

In [4]:
class CustomMarkowitzDelegate(GeneralOptimizationDelegate):
    def setup(self, optimization_object: Markowitz):
        print("Setting up the Markowitz object")
        optimization_object.mode = optimization_object.CovMode.LEDOIT_WOLF # See CovMode on the Markowitz class for more options
        optimization_object.efficient_portfolio = (
            optimization_object.EfficientPortfolio.MAX_SHARPE # See EfficientPortfolio on the Markowitz class for more options
        )

        # Now we need to set the expected returns and the covariance matrix
        if optimization_object.cov_matrix is None:
            optimization_object.cov_matrix = optimization_object.get_cov_matrix()

        optimization_object.rets = expected_returns.mean_historical_return(
            optimization_object.df
        )

        return super().setup(optimization_object) # Setup can be cascaded

### Portfolio
Delegating for an `Portfolio` class is super powerful for custom rebalancing logic. You just need to extend `PortfolioDelegate`:

In [5]:
class OptRebalancingPortfolioDelegate(PortfolioDelegate):
    def rebalance(
        self, holdings: pd.Series, prices: pd.Series, target_weights: pd.Series
    ) -> pd.Series:
        diff = optimize_trades(
            holdings=holdings,
            new_target_weights=target_weights,
            prices=prices,
            min_W=0.0,
            max_W=1.0,
            external_movement=0,
        )

        new_holdings = pd.Series(diff, index=holdings.index) + holdings
        return new_holdings

## Setting up the backtesting parameters
Backtesting plays a central role in this package. Various parameters can be set to customize the backtesting process. The following parameters can be set:

In [6]:
initial_bid = 1000 # We start with 1000 USD

max_weight = {"*": 1.0} # Can be: {'*': 1.0} or {'*': 0.5, 'BTC': 0.15}

min_weight = { '*': 0.0} # Can be: {'*': 0.0} or {'*': 0.5, 'BTC': 0.05}

budget = {} # Can be: {'uni': 1.2} for extra 20% for UNI

yield_data = pd.Series()
for asset in asset_list[asset_class]:
    yield_data[asset] = 0.075


lambda_var = 0.1
lambda_u = 0.1

rebalance_frequency = "1W"
adjust_holdings = True # Wether to adjust the holdings daily to obtain the target weights

## Portfolio Construction & backtesting
Constructing a portfolio is done by using the `Portfolio` class. The `Portfolio` class is initialized with the following parameters:
- **base_value**: The base value of the portfolio. This is the value of the portfolio at the start of the backtesting period.
- **initial_prices**: The initial prices of the assets in the portfolio. This is used to calculate the initial weights of the assets in the portfolio.
- **optimiser**: The chosen optimiser. Make sure to `bind` the delegate to the optimiser.
- **max_weight**: The maximum weight of an asset in the portfolio. This is used to set the bounds of the optimisation problem.
- **min_weight**: The minimum weight of an asset in the portfolio. This is used to set the bounds of the optimisation problem.
- **budget**: The budget of the portfolio. This is used with `RiskParity`
- **lambda_var**: The variance risk budgeting lambda parameter. This is used with `RiskParity`
- **lambda_mean**: The mean risk budgeting lambda parameter. This is used with `RiskParity`


The `Backtesting` object can be constructed as follows, where all the `Portfolio` parameters are passed as a dictionary.

In [7]:
portfolio_markowitz = Portfolio(
    base_value=initial_bid,
    initial_prices=df.loc[:start_date_portfolio],
    optimiser=Markowitz.bind(CustomMarkowitzDelegate()),
    max_weight=max_weight,
    min_weight=min_weight,
    budget=budget,
    lambda_var=lambda_var,
    lambda_u=lambda_u,
)

portfolio_markowitz.delegate = OptRebalancingPortfolioDelegate() # We can set a delegate to the portfolio

backtest = Backtest(
    portfolios={
        "Markowitz": portfolio_markowitz,
    },
    start_date=start_date_portfolio,
    end_date=df.index[-1],
    rebalance_frequency=rebalance_frequency,
    adjust_holdings=adjust_holdings,
    data=df,
    mcaps=mcaps,
    asset_class=asset_class,
)

Setting up the Markowitz object


#### Running the backtest and Saving the data
Running the backtest is done by calling the `run_backtest` method. This method takes the following parameters:
- **look_back_period**: A number of "unit" to look back at every rebalancing
- **look_back_unit**: The unit of the look back period. This can be `D` for days, `W` for weeks, `M` for months or `Y` for years.
- **yield_data**: A `pd.Series` containing the yield of the portfolio. This is APR.

In [8]:
perfs = backtest.run_backtest(
    look_back_period=120,
    look_back_unit="D",
    yield_data=yield_data,
)

# Check if the directory exists, if not, create it
if not os.path.exists(f"./out/{rebalance_frequency}_parity/"):
    os.makedirs(f"./out/{rebalance_frequency}_parity/")

backtest.export_results(
    perfs, f"./out/{rebalance_frequency}_parity/", f"backtest_results_{asset_class}.xlsx"
)

Setting up the Markowitz object
[ 8.42804879e-05  6.11319634e-10  1.10805536e-08  2.83656939e-06
 -4.39028204e-05  4.51228550e-07  1.03043184e-08  9.67672393e-09
  1.25009489e-07 -4.39272839e-05  4.80326124e-08  2.67233043e-08]
optimal_inaccurate
Setting up the Markowitz object


/Users/arguiot/Developer/Pyratz/portfolio/env/lib/python3.10/site-packages/cvxpy/problems/problem.py:1387: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
/Users/arguiot/Developer/Pyratz/portfolio/env/lib/python3.10/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:441: UserWarning: The risk_free_rate provided to portfolio_performance is different to the one used by max_sharpe. Using the previous value.
  warnings.warn(
/Users/arguiot/Developer/Pyratz/portfolio/env/lib/python3.10/site-packages/cvxpy/problems/problem.py:1387: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
/Users/arguiot/Developer/Pyratz/portfolio/env/lib/python3.10/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:441: UserWarning: The risk_free_rate provided to portfolio_perfo

[ 1.86437879e-05  1.27526595e-10  2.59050339e-09  5.11735130e-07
 -9.65777992e-06  1.05406269e-07  2.29345639e-09  2.09795990e-09
  2.88264708e-08 -9.66230761e-06  1.09990869e-08  5.86429923e-09]
optimal_inaccurate
Setting up the Markowitz object
[-4.52225703e-12 -9.13333814e-17 -1.68278180e-15 -3.37594631e-13
 -8.51066601e-06  8.51067542e-06 -1.49408143e-15 -1.55453642e-15
 -1.55553173e-14 -2.52337929e-16 -3.93569936e-15 -3.52368610e-15]
optimal


Setting up the Markowitz object
[-4.79596098e-12 -9.42984884e-17 -1.76663567e-15 -3.50046902e-13
 -8.12740214e-06  8.12741211e-06 -1.65589152e-15 -1.60764622e-15
 -1.68327291e-14 -2.51626499e-16 -3.69226778e-15 -3.70034651e-15]
optimal
Setting up the Markowitz object
[-2.15963585e-12 -4.11416142e-17 -9.37715878e-16 -1.58187530e-13
 -4.65483396e-06  4.65483845e-06 -7.10130115e-16 -6.95492249e-16
 -8.08670235e-15 -1.11082585e-16 -1.85905139e-15 -1.67268399e-15]
optimal
Setting up the Markowitz object


/Users/arguiot/Developer/Pyratz/portfolio/env/lib/python3.10/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:441: UserWarning: The risk_free_rate provided to portfolio_performance is different to the one used by max_sharpe. Using the previous value.
  warnings.warn(
/Users/arguiot/Developer/Pyratz/portfolio/env/lib/python3.10/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:441: UserWarning: The risk_free_rate provided to portfolio_performance is different to the one used by max_sharpe. Using the previous value.
  warnings.warn(
/Users/arguiot/Developer/Pyratz/portfolio/env/lib/python3.10/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:441: UserWarning: The risk_free_rate provided to portfolio_performance is different to the one used by max_sharpe. Using the previous value.
  warnings.warn(
/Users/arguiot/Developer/Pyratz/portfolio/env/lib/python3.10/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:441: UserWarning: The risk_fre

...

[-5.44625414e-06  4.73833130e-10 -4.89420866e-06  9.62484885e-06
  2.16622323e-09  4.78394934e-07  8.06047559e-09  7.76919684e-09
  1.15094110e-07  9.94888051e-10  3.59735655e-08  1.74944025e-08]
optimal
Setting up the Markowitz object
[ 7.00645663e-07 -6.16360663e-18 -7.00644949e-07 -3.99070716e-14
 -2.86973905e-17 -5.27104369e-15 -1.07064053e-16 -1.00943477e-16
 -1.57531631e-15 -1.30345359e-17 -4.87478500e-16 -2.22801645e-16]
optimal
Setting up the Markowitz object
[-3.25961289e-06  3.73544686e-10 -3.37127871e-06  9.44871604e-06
  1.72063614e-09  3.49546869e-07  6.57265351e-09  6.04734143e-09
 -3.26794520e-06  7.84552539e-10  3.17580802e-08  1.36467360e-08]
optimal
Setting up the Markowitz object
[-9.71914434e-06 -1.55104205e-09  2.12254852e-05  8.11644023e-06
 -7.09450658e-09 -8.61278928e-07 -2.60031972e-08 -2.41294039e-08
 -3.53123584e-07 -3.37903277e-09 -1.81179124e-05 -6.09347712e-08]
optimal
Setting up the Markowitz object


/Users/arguiot/Developer/Pyratz/portfolio/env/lib/python3.10/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:441: UserWarning: The risk_free_rate provided to portfolio_performance is different to the one used by max_sharpe. Using the previous value.
  warnings.warn(
/Users/arguiot/Developer/Pyratz/portfolio/env/lib/python3.10/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:441: UserWarning: The risk_free_rate provided to portfolio_performance is different to the one used by max_sharpe. Using the previous value.
  warnings.warn(
/Users/arguiot/Developer/Pyratz/portfolio/env/lib/python3.10/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:441: UserWarning: The risk_free_rate provided to portfolio_performance is different to the one used by max_sharpe. Using the previous value.
  warnings.warn(
/Users/arguiot/Developer/Pyratz/portfolio/env/lib/python3.10/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:441: UserWarning: The risk_fre

[-8.38376167e-13 -7.70963263e-18 -3.43570625e-06 -4.85529643e-14
 -3.46265366e-17 -6.44710643e-15 -1.27498404e-16 -1.16541010e-16
  1.41650740e-06 -1.61471316e-17  2.01920059e-06 -2.85438819e-16]
optimal
Setting up the Markowitz object
[ 9.80184870e-06  2.85952839e-11 -5.01979338e-06  1.86950524e-07
  1.23764818e-10  2.38227447e-08  4.58331326e-10  4.31960664e-10
  7.73239652e-09  6.34148321e-11 -5.00604404e-06  1.06622234e-09]
optimal


/Users/arguiot/Developer/Pyratz/portfolio/env/lib/python3.10/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:441: UserWarning: The risk_free_rate provided to portfolio_performance is different to the one used by max_sharpe. Using the previous value.
  warnings.warn(
/Users/arguiot/Developer/Pyratz/portfolio/env/lib/python3.10/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:441: UserWarning: The risk_free_rate provided to portfolio_performance is different to the one used by max_sharpe. Using the previous value.
  warnings.warn(
/Users/arguiot/Developer/Pyratz/portfolio/portfolio_optimization/backtesting/Backtesting.py:322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  value["Daily Return"] = daily_return


: 